In [1]:
include("AuxiliaryFunctions/BornModel.jl")
include("AuxiliaryFunctions/Plotting.jl")

import .BornModel as BM
import .Plotting as P
using Plots, Plots.PlotMeasures
using DifferentialEquations
using BifurcationKit, Parameters

┌ Warning: Error requiring `KrylovKit` from `LinearSolve`
│   exception = (LoadError("C:\\Users\\kvdhe\\.julia\\packages\\LinearSolve\\qCLK7\\ext\\LinearSolveKrylovKitExt.jl", 1, ArgumentError("Package LinearSolve does not have KrylovKit in its dependencies:\n- You may have a partially installed environment. Try `Pkg.instantiate()`\n  to ensure all packages in the environment are installed.\n- Or, if you have LinearSolve checked out for development and have\n  added KrylovKit as a dependency but haven't updated your primary\n  environment's manifest file, try `Pkg.resolve()`.\n- Otherwise you may need to report an issue with LinearSolve")), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x0000021ffe209714, Ptr{Nothing} @0x0000021ffe20998a, Ptr{Nothing} @0x00007ff8d9066a24, Ptr{Nothing} @0x00007ff8d906861e, Ptr{Nothing} @0x00007ff8d9067ddc, Ptr{Nothing} @0x00007ff8d9067f96, Ptr{Nothing} @0x00007ff8d906920e, Ptr{Nothing} @0x00007ff8b7782aba, Ptr{Nothing} @0x00007ff8b75d3b02, Ptr{N

In [2]:
# plotting
xmin, xmax = 33.5, 35.5
ymin, ymax = 17, 30

(17, 30)

## Continuation: first 1dim, then codim2

In [4]:
parsS2 = (T2 = BM.nondimensional_T(10.), S2 = BM.nondimensional_S(34.), T4 = BM.nondimensional_T(4.), S4 = BM.nondimensional_S(34.9), T0 = BM.nondimensional_T(1.6), Tamp = 0,
    η = 1.29e2, μ1 = 0.838, μ2 = 83.8, μ3 = 12.1, μ4 = 8.84e-3, r = 0.0714)

x0S2 = [BM.nondimensional_T(6), BM.nondimensional_S(34.5), BM.nondimensional_T(5), BM.nondimensional_S(34.9)]
tspan = (0., 50.)

odeprobS2 = ODEProblem(BM.nondimensional_born!, x0S2, tspan, parsS2)
odesolS2 = solve(odeprobS2)

x0S2 = odesolS2[end];

In [5]:
recordFromSolution(x, p) = (T1 = x[1], S1 = x[2], T3 = x[3], S3 = x[4],
    M = parsS2.r*(1 - parsS2.η*((parsS2.S4 - 1) - (x[4] - x[3])) - parsS2.η*parsS2.r*((p - parsS2.T2) - (x[2] - x[1]))) + (1 - parsS2.η*((parsS2.S4 - 1) - (x[4] - x[3]))),
    Δσ13 = (x[2] - x[1]) - (x[4] - x[3]),
    Δσ21 = (p  - parsS2.T2) - (x[2] - x[1]),
    Δσ43 = (parsS2.S4  - parsS2.T4) - (x[4] - x[3]),
    Ud = 1 - parsS2.η*((parsS2.S4 - 1) - (x[4] - x[3])),
    Us = 1 - parsS2.η*((parsS2.S4 - 1) - (x[4] - x[3])) - parsS2.η*parsS2.r*((p - parsS2.T2) - (x[2] - x[1]))
    )

probS2 = BifurcationProblem(BM.nondimensional_born!, x0S2, parsS2, (@lens _.S2); 
    record_from_solution = recordFromSolution
)    

# continuation
opts_newton_c = NewtonPar(tol = 1e-11, max_iterations = 50)
opts_c = ContinuationPar(p_min = BM.nondimensional_S(0), p_max = BM.nondimensional_S(50),
    dsmin=1e-10, dsmax=1e-5, ds=1e-6,
    newton_options = opts_newton_c,
    max_steps = 5e4)

brS2 = continuation(probS2, PALC(), opts_c; 
    detect_bifurcation = 3, 
    bothside = true,
    normC = norminf,
    usedeflation = true)

# diagram
opts_newton_d = NewtonPar(tol = 1e-11, max_iterations = 50)
opts_d = ContinuationPar(p_min = BM.nondimensional_S(0), p_max = BM.nondimensional_S(50),
    dsmin=1e-10, dsmax=1e-6, ds=1e-7,
    newton_options = opts_newton_d,
    max_steps = 1e6) 

dS2 = bifurcationdiagram(probS2, PALC(),
    5,
    bothside = true,
    (args...) -> opts_d,
)

[Bifurcation diagram]
 ┌─ From 0-th bifurcation point.
 ├─ Children number: 0
 └─ Root (recursion level 1)
      ┌─ Curve type: EquilibriumCont
      ├─ Number of points: 1678901
      ├─ Type of vectors: Vector{Float64}
      ├─ Parameter S2 starts at 0.0, ends at 0.01265963541810787
      ├─ Algo: PALC
      └─ Special points:

If `br` is the name of the branch,
ind_ev = index of the bifurcating eigenvalue e.g. `br.eig[idx].eigenvals[ind_ev]`

- #  1, endpoint at S2 ≈ +0.00000000,                                                                      step =   0
- #  2,       bp at S2 ≈ +0.88400540 ∈ (+0.88400540, +0.88400540), |δp|=1e-10, [converged], δ = ( 1,  0), step = 698879, eigenelements in eig[698880], ind_ev =   1
- #  3,       bp at S2 ≈ +0.85751433 ∈ (+0.85751433, +0.85751433), |δp|=5e-11, [converged], δ = ( 1,  0), step = 717737, eigenelements in eig[717738], ind_ev =   2
- #  4,     hopf at S2 ≈ +0.85759615 ∈ (+0.85759613, +0.85759615), |δp|=2e-08, [converged], δ = (-2, -2)

In [6]:
opts_newton_cd2 = NewtonPar(tol = 1e-9, max_iterations = 50)

opts_cd2 = ContinuationPar(p_min = 0., p_max = BM.nondimensional_F(4),
    dsmin=1e-8, dsmax=1e-6, ds=1e-7,
    newton_options = opts_newton_cd2,
    max_steps = 5e4) 

cd2_bp1 = continuation(dS2.γ, 2, (@lens _.μ4), opts_cd2,
	normC = norminf,
	bothside = true,
	# detection of codim 2 bifurcations with bisection
	detect_codim2_bifurcation = 2,
	start_with_eigen = false,
	update_minaug_every_step = 1,
	# we save the different components for plotting
	# record_from_solution = recordFromSolution
	)

cd2_bp2 = continuation(dS2.γ, 3, (@lens _.μ4), opts_cd2,
	normC = norminf,
	bothside = true,
	# detection of codim 2 bifurcations with bisection
	detect_codim2_bifurcation = 2,
	start_with_eigen = false,
	update_minaug_every_step = 1,
	# we save the different components for plotting
	# record_from_solution = recordFromSolution
	)

cd2_ho1 = continuation(dS2.γ, 4, (@lens _.μ4), opts_cd2,
	normC = norminf,
	bothside = true,
	# detection of codim 2 bifurcations with bisection
	detect_codim2_bifurcation = 2,
	start_with_eigen = false,
	update_minaug_every_step = 1,
	# we save the different components for plotting
	# record_from_solution = recordFromSolution
	)

cd2_bp3 = continuation(dS2.γ, 5, (@lens _.μ4), opts_cd2,
    normC = norminf,
    bothside = true,
    # detection of codim 2 bifurcations with bisection
    detect_codim2_bifurcation = 2,
    start_with_eigen = false,
    update_minaug_every_step = 1,
    # we save the different components for plotting
    # record_from_solution = recordFromSolution
    )

 ┌─ Curve type: FoldCont
 ├─ Number of points: 26914
 ├─ Type of vectors: Vector{Float64}
 ├─ Parameter μ4 starts at 0.0, ends at 0.03535991340429371
 ├─ Algo: PALC
 └─ Special points:

If `br` is the name of the branch,
ind_ev = index of the bifurcating eigenvalue e.g. `br.eig[idx].eigenvals[ind_ev]`

- #  1, endpoint at μ4 ≈ +0.00000000,                                                                      step =   0
- #  2, endpoint at μ4 ≈ +0.03535991,                                                                      step = 26913


## Plot

In [16]:
linewidth = 1.5
# titlefontsize = 12 
legendfontsize = 12

12

In [17]:
plot(dpi = 300, grid = false, legend = :topleft, legendfontsize = legendfontsize, 
    left_margin = 15px, right_margin = 10px, size = (width = 660, height = 500))

bp1_x, bp1_y = BM.dimensional_F(cd2_bp1.param), BM.dimensional_S(getproperty(cd2_bp1, :S2))
bp2_x, bp2_y = BM.dimensional_F(cd2_bp2.param), BM.dimensional_S(getproperty(cd2_bp2, :S2))
ho_x, ho_y = BM.dimensional_F(cd2_ho1.param), BM.dimensional_S(getproperty(cd2_ho1, :S2))
bp3_x, bp3_y = BM.dimensional_F(cd2_bp3.param), BM.dimensional_S(getproperty(cd2_bp3, :S2))


# plot range of bistability
# plot!(bp1_x, zeros(length(bp1_x)), fillrange = bp2_y, c = :orange, alpha = 0.2)
# plot!(bp1_x, bp1_y, fillrange = bp2_y, c = :grey, alpha = 0.2, label = "region of bistability")

# plot upper bound (lower saddle node)
plot!(bp1_x, bp1_y, c= :blue, linewidth = linewidth, label = "saddle node bifurcation")

# plot upper bound (higher saddle node)
plot!(bp2_x, bp2_y, c = :blue, linewidth = linewidth, label = "")
plot!(ho_x, ho_y, c = :red, ls = :dash, linewidth = linewidth, label = "Hopf bifurcation")
plot!(bp3_x, bp3_y, c = :blue, linewidth = linewidth, label = "")

annotate!(1.75, 33.85, "III")
annotate!(1.75, 34.7, "II")
annotate!(1.75, 35.45, "I")


xlims!(0.01, 2)
xlabel!("F [m yr\$^{-1}\$]")

ylims!(33.51, 35.5)
ylabel!("S\$_2\$ [psu]")

savefig("Figures/codim2.pdf")

fs = 10
# F = 0.5, S2 = 35
scatter!([0.5], [35], color = :black, m = :circle, label = "")
annotate!([0.55], [34.94], text("A", fs))
# F = 1, S2 = 35
scatter!([1], [35], color = :black, m = :circle, label = "")
annotate!([1.05], [34.94], text("B", fs))
# F = 1.5, S2 = 35
scatter!([1.5], [35], color = :black, m = :circle, label = "")
annotate!([1.55], [34.94], text("C", fs))
# F = 0.5, S2 = 34.75
scatter!([0.5], [34.75], color = :black, m = :circle, label = "")
annotate!([0.55], [34.69], text("D", fs))
# F = 1, S2 = 34.75
scatter!([1], [34.75], color = :black, m = :circle, label = "")
annotate!([1.05], [34.69], text("E", fs))
# F = 1.5, S2 = 34.75
scatter!([1.5], [34.75], color = :black, m = :circle, label = "")
annotate!([1.55], [34.69], text("F", fs))

savefig("Figures/codim2-withpoints.pdf")


"c:\\Users\\kvdhe\\Documents\\a. Studie\\6 - MSc 2\\Thesis\\Paper\\Code\\Figures\\codim2-withpoints.pdf"